# Import Needed Libraries

In [8]:
# To import configurations from config.ini files
import configparser
# For dataframe processes
import pandas as pd
import numpy as np

# For vizualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix

# To display all columns
pd.set_option('display.max_columns', None)

# To create a quick model to look at Feature Importances
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

# To save csv's with current date information
import datetime as dt

# Importing Configuration

In [9]:
# import and read my config.ini file
config = configparser.ConfigParser()
config.read("../src/config.ini")

['../src/config.ini']

In [10]:
# Import my output path for saving data
output = config['paths']['data_path']

# Importing Given Data
I will import the test data again later on. This is because I figure out what 
dtypes I should cast each column. I then convert this to a dictionary that can
serve as an input in *pd.read_csv()*.

In [11]:
# Import training and testing sets
train_lbls = pd.read_csv(config['paths']['train_labels'])
train_df = pd.read_csv(config['paths']['train_data'])
test_df = pd.read_csv(config['paths']['test_data'])

# Import submission format
sub_form = pd.read_csv(config['paths']['sub_form'])

Viewing the shapes and heads of each dataset to see what I'm working with

In [12]:
# Checking the shape of each dataframe
print('train_values:', train_df.shape)
print('train_labels', train_lbls.shape)
print('test_df', test_df.shape)
print('sub_form', sub_form.shape)

train_values: (59400, 40)
train_labels (59400, 2)
test_df (14850, 40)
sub_form (14850, 2)


In [13]:
train_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [14]:
train_lbls.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [15]:
test_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             14850 non-null  float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13981 non-null  object 
 4   gps_height             14850 non-null  int64  
 5   installer              13973 non-null  object 
 6   longitude              14850 non-null  float64
 7   latitude               14850 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

According to the submission format my submissions will take the form of each 
ID and the corresponding status group classification (*functional*, *non-functional*,
or *functional,needs repair*)

In [17]:
sub_form.head()

,id,status_group
0,50785,predicted label
1,51630,predicted label
2,17168,predicted label
3,45559,predicted label
4,49871,predicted label


# Data Cleaning

Good practice to check for duplicates in rows and columns

In [18]:
# Good practice to drop duplicates
# Drop row duplicates, if present
train_df.drop_duplicates(inplace=True)
# Drop column duplicates, if present
train_df = train_df.loc[:,~train_df.columns.duplicated()]

# Check if shape changed, which it did not
train_df.shape

(59400, 40)

I choose to set the dataframe index to *id* and then remove the *id* column. This
is so I can preserve id numbers without including the *id* data in my cleaning
processes. I do this for my training data and the labels. I do this after checking
for duplicates since it is possible more than 1 *id* can have the same values.

In [19]:
train_df.index = train_df['id']
train_df.drop(columns='id', inplace=True)

train_lbls.index = train_lbls['id']
train_lbls.drop(columns='id', inplace=True)

I see there are null values in my data set, so I will take a closer look at them
to decide what I can do. I also see that there are only three dtypes: 
float64, int64, and object. My first thought would to be see if I can cast any to
another dtype. This is so I have columns in a more efficient dtype and to save memory. However, this can be an optional processes considering that the dataset is very
small (~18 MB).


In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

Looks like I can cast columns to simpler dtypes and choose
the specific dtypes I want for other columns.
I drop *date_recorded* since values make <1% of the data. I'm sure I can
get some use out of this data, but I don't feel like messing with datetimes at
the moment.

In [21]:
train_df.drop(columns='date_recorded', inplace=True)

Below a function that I found to cast dtypes down. I modified the function to cast
specific columns to a specific dtype. Sadly, I lost the link to the post I found 
this code in :(

In [22]:
train_df.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [23]:
def memory_reduction(df):
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            if col == 'date_recorded':
                df[col] = pd.to_datetime(df[col])
            elif col_type == object:
                df[col] = df[col].astype('category')
            elif col in ['permit', 'public_meeting']:
                df[col] = df[col].astype(int)
            elif col == 'amount_tsh':
                df[col] = df[col].astype(np.int32)
            elif col == 'scheme_name':
                df[col] = df[col].apply(lambda x: str(x))
            else:
                continue

In [24]:
memory_reduction(train_df)

The training data is now reduced to 5.8 MB. This is about 1/3 of the space that
it used before I applied the function.

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   amount_tsh             59400 non-null  float32 
 1   funder                 55765 non-null  category
 2   gps_height             59400 non-null  int16   
 3   installer              55745 non-null  category
 4   longitude              59400 non-null  float16 
 5   latitude               59400 non-null  float16 
 6   wpt_name               59400 non-null  category
 7   num_private            59400 non-null  int16   
 8   basin                  59400 non-null  category
 9   subvillage             59029 non-null  category
 10  region                 59400 non-null  category
 11  region_code            59400 non-null  int8    
 12  district_code          59400 non-null  int8    
 13  lga                    59400 non-null  category
 14  ward                   59400 non-n

I used the training dataset dtypes to create a dtype dictionary. Thus, I can 
import the testing dataset directly as the specified dtypes. I came back to this
step and removed specific columns from the dtype dictionary due to errors. It was
just easier to import these as their default type and then cast them separately.

In [26]:
dtype_dict = train_df.dtypes.apply(lambda c: c.name).to_dict()

del dtype_dict['scheme_name']
del dtype_dict['public_meeting']
del dtype_dict['permit']

In [27]:
test_df = pd.read_csv(config['paths']['test_data'], dtype=dtype_dict)

I apply some of the same cleaning processes as before.

In [28]:
test_df.index = test_df['id']
test_df.drop_duplicates(inplace=True)
test_df = test_df.loc[:,~test_df.columns.duplicated()]
test_df.drop(columns=['id', 'date_recorded'], inplace=True)
test_df.shape

(14850, 38)

Using *.info()* I can see that I imported the test set to my specified dtypes. I
can also see that I saved memory this way as well. It now uses 2 MB compared to 
the 4.5 MB from before.

In [29]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 50785 to 68707
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   amount_tsh             14850 non-null  float32 
 1   funder                 13981 non-null  category
 2   gps_height             14850 non-null  int16   
 3   installer              13973 non-null  category
 4   longitude              14850 non-null  float16 
 5   latitude               14850 non-null  float16 
 6   wpt_name               14850 non-null  category
 7   num_private            14850 non-null  int16   
 8   basin                  14850 non-null  category
 9   subvillage             14751 non-null  category
 10  region                 14850 non-null  category
 11  region_code            14850 non-null  int8    
 12  district_code          14850 non-null  int8    
 13  lga                    14850 non-null  category
 14  ward                   14850 non-n

# Feature Engineering + Null Values
Now I will handle missing values in the data and transform/engineer some colums.
Luckily for me, missing values are in the same columns for training and testing sets.

I first check which columns are missing data in both my training sets and testing sets.

In [30]:
# append column to a list if there are any nulls in the column
null_cols = [[c,train_df[c].isnull().sum()/train_df.shape[0]] for c in train_df.columns if train_df[c].isnull().any()]
# show columns. They all appear to be categorical or binomial
null_cols

[['funder', 0.0611952861952862],
 ['installer', 0.061531986531986535],
 ['subvillage', 0.0062457912457912455],
 ['public_meeting', 0.05612794612794613],
 ['scheme_management', 0.06526936026936027],
 ['scheme_name', 0.4741750841750842],
 ['permit', 0.05144781144781145]]

In [31]:
# append column to a list if there are any nulls in the column
null_cols_test = [[c,test_df[c].isnull().sum()/test_df.shape[0]] for c in test_df.columns if test_df[c].isnull().any()]
# show columns. They all appear to be categorical or binomial
null_cols_test

[['funder', 0.05851851851851852],
 ['installer', 0.05905723905723906],
 ['subvillage', 0.006666666666666667],
 ['public_meeting', 0.055286195286195286],
 ['scheme_management', 0.06525252525252526],
 ['scheme_name', 0.4775757575757576],
 ['permit', 0.04962962962962963]]

I drop *scheme_name* since it is missing almost 50% of the data. Each category in
this column only makes up for <1% of the data

In [32]:
train_df.drop(columns=['scheme_name'], inplace=True)
test_df.drop(columns=['scheme_name'], inplace=True)

I add the categories *Unknown* and *Other* to the specified columns. I am doing
this because I will impute missing data with these values. I will also edit these
columns depending on the number of categories contained within each column.

In [33]:
for col in ['funder', 'installer', 'subvillage', 'scheme_management']:
    if col=='scheme_management':
        train_df[col] = train_df[col].cat.add_categories('Unknown')
        test_df[col] = test_df[col].cat.add_categories('Unknown')
    else:
        train_df[col] = train_df[col].cat.add_categories('Other')
        test_df[col] = test_df[col].cat.add_categories('Other')

In [34]:
train_df.fillna({'funder':'Other',
           'installer': 'Other',
           'subvillage': 'Other', 
           'public_meeting': False,
           'scheme_management': 'Unknown',
           'permit': False}
           , inplace=True)

In [35]:
test_df.fillna({'funder':'Other',
           'installer': 'Other',
           'subvillage': 'Other', 
           'public_meeting': False,
           'scheme_management': 'Unknown',
           'permit': False}
           , inplace=True)

Checking if I got all missing data

In [36]:
train_df.isna().any().sum()

0

In [37]:
test_df.isna().any().sum()

0

# Quick Model
I create a quick model here so that I can get an idea of feature importance. 
I can use the information from this step to remove any unecessary columns. I choose
a random forest model here since that is the model I want to use.

Split my data into categorical and numerical variables

In [38]:
cats = train_df.select_dtypes(include='category')
nums = train_df.select_dtypes(exclude='category')

In [39]:
cat_dict = {}
for col in cats:
    cat_dict[col] = len(cats[col].value_counts())

In [40]:
category_counts = {k: v for k, v in sorted(cat_dict.items(), key=lambda item: item[1], reverse=True)}
category_counts

{'wpt_name': 37400,
 'subvillage': 19288,
 'installer': 2146,
 'ward': 2092,
 'funder': 1898,
 'lga': 125,
 'region': 21,
 'extraction_type': 18,
 'scheme_management': 13,
 'extraction_type_group': 13,
 'management': 12,
 'source': 10,
 'basin': 9,
 'water_quality': 8,
 'extraction_type_class': 7,
 'payment': 7,
 'payment_type': 7,
 'source_type': 7,
 'waterpoint_type': 7,
 'quality_group': 6,
 'waterpoint_type_group': 6,
 'management_group': 5,
 'quantity': 5,
 'quantity_group': 5,
 'source_class': 3,
 'public_meeting': 2,
 'permit': 2,
 'recorded_by': 1}

I label encode my categoricals since the model requires me to do so. I choose to
do label encoding since it's fast and I will have less columns to look through.

In [41]:
for col in cats.columns:
    le = LabelEncoder()
    cats[col] = le.fit_transform(cats[col])


In [42]:
train_df_exp = cats.merge(nums, left_index=True, right_index=True)

I fit my data through a random forest model. Then I put the results into a dictionary
that I sort by value, descending.

In [43]:
rf = RandomForestClassifier(random_state=42)
rf.fit(train_df_exp, train_lbls)
importances = dict(zip(train_df_exp.columns, rf.feature_importances_))
importances = {k: v for k, v in sorted(importances.items(), key=lambda item: item[1], reverse=True)}
importances

C:\Users\Cristian\AppData\Local\Temp\ipykernel_17216\1299335176.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_df_exp, train_lbls)


{'latitude': 0.08181526878002009,
 'wpt_name': 0.0760373661646006,
 'quantity': 0.07365315338001982,
 'subvillage': 0.06540170991187305,
 'quantity_group': 0.06431780544622547,
 'longitude': 0.06141382401797456,
 'gps_height': 0.05483337193156758,
 'ward': 0.04162021020133068,
 'construction_year': 0.040097765868670504,
 'population': 0.03714835116065888,
 'waterpoint_type': 0.03503917149503383,
 'funder': 0.03450521455694876,
 'waterpoint_type_group': 0.029306038314549396,
 'installer': 0.027228593454135867,
 'lga': 0.024032228779419507,
 'extraction_type_class': 0.022020162098608673,
 'amount_tsh': 0.019323809264428064,
 'payment': 0.017621442087186683,
 'extraction_type_group': 0.01689864749420747,
 'district_code': 0.015827429370687814,
 'extraction_type': 0.015396771387403837,
 'region': 0.01473103296745933,
 'payment_type': 0.014655976757191833,
 'region_code': 0.01407618330886256,
 'source': 0.01348145486684026,
 'management': 0.013267788945915808,
 'scheme_management': 0.012990

I generate a correlation matrix and then make it an upper triangle for easier
visualization.

In [44]:
corr_matrix = train_df_exp.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper


C:\Users\Cristian\AppData\Local\Temp\ipykernel_17216\2605475782.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
funder,NaN,0.547373,0.001516,0.132456,0.021835,0.150219,0.029671,0.011640,0.029277,NaN,0.009799,0.057965,0.022661,0.010749,0.024928,0.051476,0.026953,0.039548,0.048716,0.034334,0.024917,0.044186,0.044186,0.085366,0.092872,0.041615,0.050060,0.035129,0.003413,0.013124,0.037430,0.027201,0.010734,0.017316,0.018477,0.010240,0.099034
installer,NaN,NaN,0.001714,0.102048,0.024565,0.141996,0.130360,0.002677,0.004935,NaN,0.026320,0.049227,0.003649,0.018917,0.057793,0.017739,0.042735,0.004430,0.022377,0.015132,0.008955,0.054181,0.054181,0.101240,0.094940,0.058344,0.021476,0.011787,0.010294,0.006660,0.009650,0.001580,0.013550,0.018703,0.023926,0.011890,0.039931
wpt_name,NaN,NaN,NaN,0.016780,0.091068,0.043538,0.037952,0.000520,0.018770,NaN,0.065432,0.051100,0.003091,0.000737,0.019227,0.051926,0.025429,0.047708,0.056838,0.014150,0.017761,0.020468,0.020468,0.001761,0.000277,0.013852,0.026533,0.023626,0.014664,0.055669,0.017383,0.062242,0.002700,0.011861,0.006808,0.002178,0.008340
basin,NaN,NaN,NaN,NaN,0.057696,0.114880,0.006718,0.043157,0.017717,NaN,0.056740,0.134764,0.167450,0.189307,0.177939,0.061139,0.068413,0.017788,0.029961,0.075529,0.044721,0.026707,0.026707,0.061457,0.074808,0.031636,0.009864,0.013518,0.013977,0.161768,0.216101,0.218504,0.018325,0.139887,0.189692,0.066899,0.268151
subvillage,NaN,NaN,NaN,NaN,NaN,0.011206,0.009567,0.066010,0.030165,NaN,0.009136,0.032460,0.011332,0.005237,0.017299,0.003358,0.003202,0.021207,0.022473,0.002076,0.013975,0.006003,0.006003,0.019990,0.025664,0.012342,0.017225,0.019984,0.000028,0.003936,0.009325,0.007442,0.011122,0.018479,0.023079,0.010631,0.046751
region,NaN,NaN,NaN,NaN,NaN,NaN,0.193919,0.051451,0.109037,NaN,0.029334,0.049820,0.217222,0.233877,0.210724,0.076552,0.023112,0.054115,0.056709,0.073382,0.129883,0.034766,0.034766,0.156225,0.140439,0.010899,0.279421,0.257183,0.019866,0.323833,0.142766,0.025485,0.037351,0.109343,0.021688,0.004416,0.141681
lga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061165,0.017387,NaN,0.052130,0.080051,0.024245,0.007504,0.002427,0.082065,0.012648,0.110791,0.182848,0.016568,0.030434,0.014416,0.014416,0.065904,0.061295,0.021990,0.030154,0.033760,0.010315,0.137375,0.227476,0.268501,0.000843,0.042477,0.110921,0.004088,0.133283
ward,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006094,NaN,0.006398,0.030837,0.002728,0.004033,0.024567,0.002292,0.045600,0.002814,0.003249,0.011279,0.001478,0.001793,0.001793,0.010146,0.021475,0.030208,0.002778,0.004128,0.000090,0.043386,0.037396,0.007590,0.014877,0.029512,0.044599,0.029582,0.078459
public_meeting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.156408,0.140349,0.078362,0.098466,0.119903,0.097260,0.218954,0.094398,0.203768,0.025907,0.079779,0.068698,0.068698,0.071273,0.050397,0.019172,0.087268,0.090392,0.025683,0.032375,0.078823,0.034384,0.011230,0.044806,0.023855,0.000398,0.007490
recorded_by,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I extract columns from the correlation matrix that have a correlation of .6 or higher.
I use the results to generate a list of columns that could potentially be dropped.

In [45]:
# Find features with correlation greater than 0.95
potential_drops = [column for column in upper.columns if any(upper[column] > 0.6)]
potential_drops

['extraction_type_group',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity_group',
 'source_type',
 'waterpoint_type_group',
 'district_code',
 'construction_year']

This is a list of columns that I'm dropping based on correlation and feature importance.
Of columns that are similar, I choose to keep the feature with the highest feature
importance. For example, I choose to keep *extraction_type_class* and drop
*extraction_type* and *extraction_type_group* even though the *class* column was
a potential column drop.

In [46]:
drop_final = ['extraction_type', 'extraction_type_group',
              'management_group',
              'payment_type',
              'quantity_group',
              'source_type','source_class', 
              'waterpoint_type_group',
              'district_code', 
              'construction_year',
              'num_private',
              'recorded_by']

Drop the columns from the training and testing sets

In [47]:
df_final = train_df.drop(columns=drop_final)
test_df = test_df.drop(columns=drop_final)

# Finish Cleaning

Now I check which categorical columns have a high number of categories. I want to
limit the number of categories since my dimensionality will blow up when I decide
to one hot encode. I currently set the limit to 20 to keep the category counts
pretty small.

In [48]:
cats_test = test_df.select_dtypes(include='category')

In [49]:
cat_dict_test = {}
for col in cats_test:
    cat_dict_test[col] = len(cats_test[col].value_counts())

In [50]:
category_counts_test = {k: v for k, v in sorted(cat_dict_test.items(), key=lambda item: item[1], reverse=True)}
category_counts_test

{'wpt_name': 10840,
 'subvillage': 8444,
 'ward': 1959,
 'installer': 1092,
 'funder': 981,
 'lga': 125,
 'region': 21,
 'scheme_management': 12,
 'management': 12,
 'source': 10,
 'basin': 9,
 'water_quality': 8,
 'extraction_type_class': 7,
 'payment': 7,
 'waterpoint_type': 7,
 'quality_group': 6,
 'quantity': 5}

In [51]:
category_counts

{'wpt_name': 37400,
 'subvillage': 19288,
 'installer': 2146,
 'ward': 2092,
 'funder': 1898,
 'lga': 125,
 'region': 21,
 'extraction_type': 18,
 'scheme_management': 13,
 'extraction_type_group': 13,
 'management': 12,
 'source': 10,
 'basin': 9,
 'water_quality': 8,
 'extraction_type_class': 7,
 'payment': 7,
 'payment_type': 7,
 'source_type': 7,
 'waterpoint_type': 7,
 'quality_group': 6,
 'waterpoint_type_group': 6,
 'management_group': 5,
 'quantity': 5,
 'quantity_group': 5,
 'source_class': 3,
 'public_meeting': 2,
 'permit': 2,
 'recorded_by': 1}

In [52]:
def lower_features(df, df2):
    for col in ['wpt_name', 'subvillage', 'installer', 'ward', 'funder', 'lga']:
        # get top 20 categories for the column
        temp = df[col].value_counts().head(20).keys()
        # if the value is not in the top 20, convert to 'Other'
        df[col] = df[col].apply(lambda x: 'Other' if x not in temp else x)
        df2[col] = df2[col].apply(lambda x: 'Other' if x not in temp else x)

In [53]:
lower_features(df_final, test_df)

*public_meeting* and *permit* are still categories for my training set, so I cast 
them to booleans here. they are booleans in my testing set.

In [54]:
df_final[['public_meeting', 'permit']] = df_final[['public_meeting', 'permit']].astype('boolean')

I convert the following columns to categories since they are objects. I do this 
so I can one hot encode later

In [55]:
df_final[['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward']] = df_final[
    ['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward']].astype('category')

In [56]:
test_df[['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward']] = test_df[
    ['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward']].astype('category')

I separate categorical and numerical values

In [57]:
cats = df_final.select_dtypes(include='category')
nums = df_final.select_dtypes(exclude='category')

In [58]:
cats_test = test_df.select_dtypes(include='category')
nums_test = test_df.select_dtypes(exclude='category')

I one hot encode my categories. I set *handle_unknown='ignore'* to make sure I 
account for any new/unknown categories. This will make sure my training and testing
sets will result in the same columns in the end.

In [59]:
ohe = OneHotEncoder(handle_unknown='ignore')
# Fit categories to training data
ohe.fit(cats)

OneHotEncoder(handle_unknown='ignore')

In [60]:
train_ohe = pd.DataFrame(ohe.transform(cats).toarray(), columns = ohe.get_feature_names(), index=cats.index)
test_ohe = pd.DataFrame(ohe.transform(cats_test).toarray(), columns = ohe.get_feature_names(), index = cats_test.index)

C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Cristian\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [61]:
train_ohe.shape

(59400, 228)

In [62]:
test_ohe.shape

(14850, 228)

Merge encoded variables and numerical variables into my final sets

In [63]:
df_final = nums.merge(train_ohe, left_index=True, right_index=True, how='inner')
test_final = nums_test.merge(test_ohe, left_index=True, right_index=True, how='inner')

# Export Data

I add current dates and times to the names of my files for version control. I include
the time since I sometimes rerun the same code more than once on the same day.

In [64]:
current_time = dt.datetime.now().strftime("%d%m%Y_%I%M%p")

Upon trying to make a model in a separate jupyter noteboook, I find that using
csv files creates errors. However, if I use a .pkl file, my code runs fine. Thus,
I will save data as .pkl files.

In [65]:
# df_final.to_csv(output+'training_set_cleaned'+current_time+'.csv')
# test_final.to_csv(output+'testing_set_cleaned'+current_time+'.csv')
# train_lbls.to_csv(output+'train_lbls_cleaned'+current_time+'.csv')

In [66]:
df_final.to_pickle(output+'training_set_cleaned'+current_time+'.pkl')
test_final.to_pickle(output+'testing_set_cleaned'+current_time+'.pkl')
train_lbls.to_pickle(output+'train_lbls_cleaned'+current_time+'.pkl')